In [2]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
from collections import Counter


In [3]:
def short_name(team):
        short_name={'Chennai Super Kings':'CSK',
    'Mumbai Indians':'MI',
    'Gujarat Titans':'GT',
    'Kolkata Knight Riders':'KKR',
    'Punjab Kings':'PBKS',
    'Sunrisers Hyderabad':'SRH',
    'Rajasthan Royals':'RR',
    'Lucknow Super Giants':'LSG',
    'Delhi Capitals':'DC',
    'Royal Challengers Bangalore':'RCB'
    }
        return short_name[team]

In [4]:
def get_moth(url):
    cricbuzz_highlights_url=url
    cricbuzz_highlights_url=cricbuzz_highlights_url.replace('cricket-full-commentary','cricket-scores')
    cricbuzz_highlights_url
    page=requests.get(cricbuzz_highlights_url)
    soup=BeautifulSoup(page.content)
    try: 
        win=soup.find('div',{'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'}).text
        #cb-col cb-col-100 cb-mini-col cb-min-comp ng-scope
        #cb-link-undrln ng-binding
        a=soup.find('span',{'class':'cb-text-gray cb-font-12'})
        try:
            man_of_the_match=a.next_sibling.next_sibling.next_sibling.text
        except:
            man_of_the_match=None
    except:
        win = None
        man_of_the_match=None
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['win']=win
    df['man_of_the_match']=man_of_the_match
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

In [5]:
def find_toss(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss=df['toss'][0]
        toss_win=toss[:toss.find('have')]
        toss_choose=toss.split(' ')[-1]

        df['Toss_winner']=short_name(toss_win.strip())
        df['toss_chosen']=toss_choose
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')
        pass
#find_toss('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [6]:

def find_innings(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
       
    try :
        toss_winner=df['Toss_winner'][0]
        #print(toss_winner)
        toss_choose=df['toss_chosen'][0]
        #print(toss_choose)
        if toss_choose=='field':
            #print('fielding')
           
            #print(df[df['innings']== f'{toss_winner} Inns'.strip()]['innings'])
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            
            #print(df)
        else:
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            
            
           
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')

#find_innings('https://www.cricbuzz.com/cricket-full-commentary/66442/csk-vs-kkr-61st-match-indian-premier-league-2023')

In [7]:
def find_bowling_team(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss_winner=df['Toss_winner'][0]
        toss_choose=df['toss_chosen'][0]
        a_team=short_name(df['team_a'][0])
        b_team=short_name(df['team_b'][0])
        teams=[a_team,b_team]
        for i in teams:
            if i==toss_winner:
                teams.remove(i)
        opt=teams[0]
        if toss_choose=='bat':
            df.loc[df['innings']=='1st Inns','batting_team']=toss_winner
            df.loc[df['innings']=='1st Inns','bowling_team']=opt
            df.loc[df['innings']=='2nd Inns','batting_team']=opt
            df.loc[df['innings']=='2nd Inns','bowling_team']=toss_winner
        if toss_choose=='field':
            df.loc[df['innings']=='1st Inns','batting_team']=opt
            df.loc[df['innings']=='1st Inns','bowling_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','batting_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','bowling_team']=opt
            
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except Exception as e:
        print(e)
       
#find_bowling_team('https://www.cricbuzz.com/cricket-full-commentary/66519/kkr-vs-gt-39th-match-indian-premier-league-2023')

In [8]:
def find_batsman_bowler(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    df['Batsman']=''
    df['Bowler']=''
    #display(df)
    for i,row in enumerate(df['Commentary Text'].values):
        text=row.split(',')[0]
        text=text.split(' to ')
        batsman=text[1]
        bowler=text[0]
        df.loc[i,['Batsman']]=batsman
        #df['Batsman'][i]=batsman
        df.loc[i,['Bowler']]=bowler

        #df['Bowler'][i]=bowler
    #display(df)
    df.to_csv(f'files/2023/{file_name}.csv',index=False)
        

#find_batsman_bowler('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [9]:
texts=[]
#find the runs by ball 
def find_runs(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['runs']=''
    df['extra']=''
    for i,comment in enumerate(df['Commentary Text'].values):
        text=comment.split(',')[1].strip().lower()
        #print('text=',text)
        texts.append(text)
        if text.startswith('1 run'.casefold()):
            df.loc[i,['runs']]= 1
            df.loc[i,['extra']]= 0
        elif text.startswith('2 runs'.casefold()):
            text= 2
            df.loc[i,['runs']]= 2
            df.loc[i,['extra']]= 0
        elif text.startswith('no run'.casefold()):
            df.loc[i,['runs']]= 0
            df.loc[i,['extra']]= 0
        elif text.startswith('3 runs'.casefold()):
            df.loc[i,['runs']]= 3
            df.loc[i,['extra']]= 0
        elif text.startswith('FOUR'.casefold()):
            df.loc[i,['runs']]= 4
            df.loc[i,['extra']]= 0
        elif text.startswith('SIX'.casefold()):
            df.loc[i,['runs']]= 6
            df.loc[i,['extra']]= 0
        elif text.startswith(('byes'.casefold(),'leg'.casefold())):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= 4
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]= 6
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=0
            df.loc[i,['runs']]= 'byes'
        elif text.startswith('no ball'.casefold()):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= '1+4'
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]='1+6'
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=1
            df.loc[i,['runs']]= 'no ball'
        elif text.startswith('out'.casefold()):
            df.loc[i,['runs']]= 'out'
            df.loc[i,['extra']]= 'out'
        elif text.startswith('wide'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 1
        elif text.startswith('2 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 2
        elif text.startswith('3 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 3
        elif text.startswith('4 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 4
        elif text.startswith('5 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 5
        
        #print(text)
    #display(df.sample(5))
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

#find_runs('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [10]:
# find the length of the each ball
def find_length(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    length=[]
    none=[]
    df=pd.read_csv(f'files/2023/{file_name}.csv')

    #display(df)
    for i,line in enumerate(df["Commentary Text"]):
        match=re.search('short|slot|full-toss|back of|good|tosse|full|on a length|yorker|of a length|length delivery|length ball|length on|length,',line,re.IGNORECASE)

        length_balls=['on a length','of a length','length delivery','length ball','length on','length,']
    
        if match:
            
            if match.group(0).casefold()=='short':
                df.loc[i,['ball_length']]= 'Short'
            elif match.group(0).casefold()=='full':
                df.loc[i,['ball_length']]= 'Full'
            elif match.group(0).casefold()=='slot':
                df.loc[i,['ball_length']]= 'Slot'
            elif match.group(0).casefold()=='back of':
                df.loc[i,['ball_length']]= 'Back of Length'
            elif match.group(0).casefold()=='good':
                df.loc[i,['ball_length']]= 'Good'
            elif match.group(0).casefold()=='tosse' or match.group(0).casefold()=='full-toss':
                df.loc[i,['ball_length']]= 'Full toss'
            elif match.group(0).casefold()=='yorker':
                df.loc[i,['ball_length']]= 'yorker'
            elif  match.group(0).casefold() in length_balls:
                df.loc[i,['ball_length']]= 'Good'
            else:
                print(match.group(0))
                break

        else:
            df.loc[i,['ball_length']]= 'undefined'
    #display(df)
    df.to_csv(f'files/2023/{file_name}.csv',index=False)    
#find_length('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [96]:
def check_commentary(url):
    file_name=url.split('/')[-1]
    k=0
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    l=0
    for i,line in enumerate(df["Commentary Text"]):
        fielding_positions = re.search(r"(?i)\b(?:gully|slip|third\s*man|keeper|hort fine|down the pitch|straight|fine\s*leg|square\s*leg|mid\s*wicket|mid-wicket|deep backward square|long-on|long\s*on|long\s*off|long-off|cover|mid\s*off|mid-off|mid\s*on|mid-on|point|deep square|deep\s*square\s*leg|deep\s*mid\s*wicket|deep\s*cover|deep\s*point|short\s*leg|silly\s*point|short\s*mid\s*wicket|backward\s*point|extra\s*cover|short\s*third\s*man|long\s*stop|leg\s*slip|short\s*fine\s*leg|short\s*mid\s*off|short\s*mid\s*on|cow\s*corner|deep\s*backward\s*point|deep\s*extra\s*cover|deep\s*mid\s*off|deep\s*mid\s*on|deep\s*silly\s*point|fine\s*leg\s*slip|fine\s*third\s*man|long\s*leg|long\s*silly\s*point|long\s*leg\s*slip|short\s*cover|short\s*extra\s*cover|short\s*long\s*leg|short\s*mid\s*leg|short\s*square\s*leg|super\s*cover|super\s*deep\s*extra\s*cover|super\s*extra\s*cover|super\s*long\s*off|super\s*long\s*on|super\s*mid\s*off|super\s*mid\s*on|super\s*point|super\s*short\s*cover|covers|super\s*square\s*leg|super\s*third\s*man|short fine|off-side|square|super\s*mid\s*wicket)\b", line, flags=re.IGNORECASE)
        match=re.search('drive',line,re.IGNORECASE)
        if fielding_positions:
            l=l+1
            # print(fielding_positions.group(0))
        else:
            if 'fine' in line.lower():
                print(line)
                k=k+1
    print(l)
    print(k)
check_commentary('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

 Chahal to Hardik Pandya, 1 run, risky! But he escapes. Pandya blocked this off the front foot to the right of the bowler, Chahal attacked with the slide and fired the throw but it was wide. Flings his left elbow in mild pain as he landed badly. Is fine to carry on 
 Chahal to Suryakumar Yadav, 1 run, goes for the sweep again, with the ball this time deflecting fine off the pad past leg stump as Chahal fires it through. The umpire gives it runs, maybe an edge was involved too 
185
2


In [12]:
def find_line(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    #print(df.info())
    l=0
    k=0
    m=0
    for i,line in enumerate(df["Commentary Text"]):
        if df.loc[i,'ball_length'] =='undefined':
            
            match=re.search('down-leg|wide|at the stump|off stump|outside-off|on the pad|outside-leg|outside off|on middle|on the stumps|the hip|sticks|into the wicket|outside leg|to off|on off|on leg|leg stump|leg-lined|middle-lined',line,re.IGNORECASE)
            if match:
                m=m+1
                #print(match)
                stumps=['at the stumps','on middle','on the stumps','the hip','sticks','into the wicket','leg stump','down leg','middle-lined','at the stump','on the pad']

                off_side=['off stump','outside-off','outside off','on off','on off','to off']
                leg_side=['outside leg','on leg','leg stump','leg-lined','outside-leg']


                if match.group(0).casefold() in stumps:
                    df.loc[i,['ball_line']]= 'stump line'
                elif match.group(0).casefold() in off_side:
                    df.loc[i,['ball_line']]= 'off side'
                elif match.group(0).casefold() in leg_side:
                    df.loc[i,['ball_line']]= 'leg side'
                elif match.group(0).casefold() == 'wide':
                    df.loc[i,['ball_line']]= 'wide' 
                else:
                    
                    print('error',match.group(0))
            else:
                k=k+1
                df.loc[i,['ball_line']]= 'undefined'
        else:
            
            match=re.search('down-leg|wide|at the stump|off stump|outside-off|outside-leg|on the pad|outside off|on middle|on the stumps|the hip|sticks|into the wicket|outside leg|to off|on off|on leg|leg stump|leg-lined|middle-lined',line,re.IGNORECASE)
            if match:
                #print(match)
                m=m+1
                stumps=['at the stump','on middle','on the stumps','the hip','sticks','into the wicket','leg stump','down leg','middle-lined','at the stump','on the pad']
                off_side=['off stump','outside-off','outside off','on off','on off','to off']
                leg_side=['outside leg','on leg','leg stump','leg-lined','outside-leg']


                if match.group(0).casefold() in stumps:
                    df.loc[i,['ball_line']]= 'stump line'
                elif match.group(0).casefold() in off_side:
                    df.loc[i,['ball_line']]= 'off side'
                elif match.group(0).casefold() in leg_side:
                    df.loc[i,['ball_line']]= 'leg side'
                elif match.group(0).casefold() == 'wide':
                    df.loc[i,['ball_line']]= 'wide' 
                else:
                    print('error',match.group(0))
            else:
                k=k+1
                
                df.loc[i,['ball_line']]= 'undefined'
    print('match=',m)
    print('non match=',k)
    #display(df)
    df.to_csv(f'files/2021/{file_name}.csv',index=False)  
#find_line('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [97]:
def find_shot_type(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    l=0
    k=0
    df=pd.read_csv(f'files/2021/{file_name}.csv')

    #display(df)
    for i,line in enumerate(df["Commentary Text"]):
        #print(line)
        fielding_positions = re.search(r"(?i)\b(?:gully|slip|third\s*man|fine\s*leg|down the pitch|square\s*leg|mid\s*wicket|mid-wicket|deep backward square|long-on|long\s*on|long\s*off|long-off|cover|mid\s*off|mid-off|mid\s*on|mid-on|point|deep square|deep\s*square\s*leg|deep\s*mid\s*wicket|deep\s*cover|deep\s*point|short\s*leg|silly\s*point|short\s*mid\s*wicket|backward\s*point|extra\s*cover|short\s*third\s*man|long\s*stop|leg\s*slip|short\s*fine\s*leg|short\s*mid\s*off|short\s*mid\s*on|cow\s*corner|deep\s*backward\s*point|deep backward square|deep\s*extra\s*cover|deep\s*mid\s*off|deep\s*mid\s*on|deep\s*silly\s*point|fine\s*leg\s*slip|fine\s*third\s*man|long\s*leg|long\s*silly\s*point|long\s*leg\s*slip|short\s*cover|short\s*extra\s*cover|short\s*long\s*leg|short\s*mid\s*leg|short\s*square\s*leg|super\s*cover|super\s*deep\s*extra\s*cover|super\s*extra\s*cover|super\s*long\s*off|super\s*long\s*on|super\s*mid\s*off|super\s*mid\s*on|super\s*point|super\s*short\s*cover|covers|super\s*square\s*leg|super\s*third\s*man|off-side|keeper|straight|short fine|square|super\s*mid\s*wicket)\b", line, flags=re.IGNORECASE)
        match=re.search('drive',line,re.IGNORECASE)
        if fielding_positions:
            l=l+1
            # print(fielding_positions.group(0))
        else:
            print(line)
            k=k+1
    print(l)
    print(k)

find_shot_type('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

 Harshal Patel to Bumrah, no run, now he almost gets a sixth with an inswinging yorker, Bumrah couldn't squeeze it. Ball was just outside off stump, AB threw the ball onto the sticks, ball ricochets and decide against stealing run 
 Harshal Patel to Marco Jansen, out Bowled!! One ball later but he has a five-wicket haul. Harshal Patel with an inswinging yorker that smashes the sticks. Wow! Kohli is delighted. All those worried about RCB's death bowling will be happy to see this. Jansen obviously not well equipped to dig this out, loses the off stick and walks back. Marco Jansen b Harshal Patel 0(2) 
 Harshal Patel to Marco Jansen, no run, oh almost bowled! Jansen backed away and missed this dipping slower ball yorker 
 Jamieson to Pollard, no run, beaten! Pollard not picking slower balls, swings early and misses 
 Jamieson to Krunal Pandya, no run, brilliant yorker and the bat is in two pieces. Pandya swung, the handle is in the hands, but the rest of the willow broke and fell to groun

In [14]:
links=[]
with open('match_links/2023_season_match_links.txt','r') as file:
    for line in file:
        links.append(line.strip())
links=links[74:]
k=1
# for i in tqdm(links,colour='green'):
#     print(k)
#     find_line(i)
#     k=k+1